# Knet RNN example
**TODO**: Add dropout?

In [1]:
using Pkg; haskey(Pkg.installed(),"Knet") || Pkg.add("Knet")
using Knet
import Knet: train!
True=true # so we can read the python params
include("common/params_lstm.py")
gpu()

0

In [2]:
println("OS: ", Sys.KERNEL)
println("Julia: ", VERSION)
println("Knet: ", Pkg.installed()["Knet"])
println("GPU: ", read(`nvidia-smi --query-gpu=name --format=csv,noheader`,String))

OS: Linux
Julia: 1.0.0
Knet: 1.1.1
GPU: Tesla K80



In [3]:
# define layers
_atype = gpu() >= 0 ? KnetArray{Float32} : Array{Float64}


mutable struct Embedding
    w
end


function Embedding(vocabsize::Int, embedsize::Int; atype=_atype)
    w = param(embedsize, vocabsize; init=xavier, atype=atype)
    return Embedding(w)
end


(l::Embedding)(x) = l.w[:, x]


mutable struct Projection
    w
end


function Projection(input_dim::Int, output_dim::Int; atype=_atype)
    w = param(output_dim, input_dim; atype=atype)
    return Projection(w)
end


(l::Projection)(x) = l.w * mat(x)

In [4]:
# define model and loss
mutable struct Model
    embed
    rnn
    fc
end


function Model(nfeatures::Int, nembed::Int, nhidden::Int; atype=_atype)
    embed = Embedding(nfeatures, nembed; atype=atype)
    rnn = RNN(nembed, nhidden; rnnType=:gru)
    fc = Projection(nhidden, 2)
    return Model(embed, rnn, fc)
end


function (model::Model)(x)
    indices = permutedims(hcat(x...))
    embed = model.embed(indices)
    hidden = model.rnn(embed)[:,:,end]
    output = model.fc(hidden)
end


loss(model,x,y)=nll(model(x), y)

loss (generic function with 1 method)

In [5]:
# helper to initialize optimizers
function initopt!(model, optim; options...)
    for par in params(model)
        par.opt = optim(; options...)
    end
end


# helper to update the weights
function train!(model::Model, x, y; pdrops=(0.25, 0.5))
    J = @diff loss(model, x, y)
    for par in params(model)
        g = grad(J, par)
        update!(value(par), g, par.opt)
    end
end

train! (generic function with 2 methods)

In [6]:
# load data
include(Knet.dir("data","imdb.jl"))
@time (xtrn,ytrn,xtst,ytst,imdbdict)=imdb(maxlen=MAXLEN,maxval=MAXFEATURES)
for d in (xtrn,ytrn,xtst,ytst); println(summary(d)); end

┌ Info: Loading IMDB...
└ @ Main /kuacc/users/ikesen16/.julia/packages/Knet/3lzCR/data/imdb.jl:57


 15.098990 seconds (29.44 M allocations: 1.496 GiB, 6.40% gc time)
25000-element Array{Array{Int32,1},1}
25000-element Array{Int8,1}
25000-element Array{Array{Int32,1},1}
25000-element Array{Int8,1}


In [7]:
imdbarray = Array{String}(undef,88584)
for (k,v) in imdbdict; imdbarray[v]=k; end
imdbarray[xtrn[1]]

150-element Array{String,1}:
 "nice"          
 "cinematography"
 "i"             
 "can"           
 "hardly"        
 "say"           
 "anything"      
 "positive"      
 "about"         
 "this"          
 "movie"         
 "the"           
 "single"        
 ⋮               
 "plot"          
 "or"            
 "the"           
 "bad"           
 "acting"        
 "disturbing"    
 "any"           
 "appeal"        
 "that"          
 "might"         
 "be"            
 "left"          

In [8]:
# prepare for training
weights = nothing; Knet.gc(); # Reclaim memory from previous run
model = Model(MAXFEATURES, EMBEDSIZE, NUMHIDDEN)
initopt!(model, Adam; lr=LR, beta1=BETA_1, beta2=BETA_2, eps=EPS);

In [9]:
# cold start
@time for (x,y) in minibatch(xtrn,ytrn,BATCHSIZE;shuffle=true)
    train!(model, x, y)
end

 21.101671 seconds (15.31 M allocations: 841.157 MiB, 16.92% gc time)


In [10]:
# prepare for training
weights = nothing; Knet.gc(); # Reclaim memory from previous run
model = Model(MAXFEATURES, EMBEDSIZE, NUMHIDDEN)
initopt!(model, Adam; lr=LR, beta1=BETA_1, beta2=BETA_2, eps=EPS);

In [11]:
# 29s
@info("Training...")
@time for epoch in 1:EPOCHS
    @time for (x,y) in minibatch(xtrn,ytrn,BATCHSIZE;shuffle=true)
        train!(model, x, y)
    end
end

┌ Info: Training...
└ @ Main In[11]:2


  9.998816 seconds (624.31 k allocations: 79.030 MiB, 7.06% gc time)
  9.217722 seconds (566.49 k allocations: 76.379 MiB, 6.39% gc time)
  9.355325 seconds (583.71 k allocations: 76.642 MiB, 7.85% gc time)
 28.573209 seconds (1.78 M allocations: 232.099 MiB, 7.10% gc time)


In [12]:
@info("Testing...")
@time accuracy(model, minibatch(xtst,ytst,BATCHSIZE))

┌ Info: Testing...
└ @ Main In[12]:1


  3.721205 seconds (2.26 M allocations: 172.522 MiB, 1.55% gc time)


0.8496794871794872